In [2]:
import pandas as pd
import numpy as np

In [39]:
registers = pd.read_csv("registers.csv", delimiter=",")
masks = pd.read_csv("masks.csv", delimiter=",")
print(registers)

print(masks)

     Identifier  Address Format   Type      Description MaskFamily
0    DigitalSet      123    U16    NaN      Sets output     DOMask
1  DigitalClear      124    U16    NaN    Clears output     DOMask
2      PwmStart      125     U8    NaN       Starts pwm    PwmMask
3       PwmStop      126     U8    NaN        Stops pwm    PwmMask
4     Heartbeat       12     U8  Event  Heartbeat event        NaN
     Identifier BitMask Format      Description
0  DOMask_Poke0    1<<0    U16  Output of Poke0
1  DOMask_Poke1    1<<1    U16  Output of Poke1
2  DOMask_Poke2    1<<2    U16  Output of Poke2
3   PwmMask_DO0    1<<0     U8    Pwm channel 0
4   PwmMask_DO1    1<<1     U8    Pwm channel 1


In [11]:
datatype_conversion = {
    "U16":"uint16_t",
    "U8":"uint8_t",
}

In [21]:
txt_fnc = lambda row: f"""	{datatype_conversion[row.Format]} {row.Identifier};
"""
registers_data = registers.apply(txt_fnc, axis=1).values.tolist()

header_register_s =\
"""//************************************************************************//
//* Registers' structure                                                 *//
//************************************************************************//
typedef struct
{
""" + f"{''.join(registers_data)}" + """
} AppRegs;
"""


//************************************************************************//
//* Registers' structure                                                 *//
//************************************************************************//
typedef struct
{
	uint16_t DigitalSet;
	uint16_t DigitalClear;
	uint8_t PwmStart;
	uint8_t PwmStop;
	uint8_t Heartbeat;

} AppRegs;



In [38]:
txt_fnc = lambda row: f"""#define {row.Identifier}	{row.Address} // {row.Format}	{row.MaskFamily if pd.isnull(row.MaskFamily) is False else ""}	{row.Description if pd.isnull(row.Description) is False else ""}
"""
registers_data = registers.apply(txt_fnc, axis=1).values.tolist()

header_register =\
"""/************************************************************************/
/* Registers' address                                                   */
/************************************************************************/
/* Registers */
""" + f"{''.join(registers_data)}"




/************************************************************************/
/* Registers' address                                                   */
/************************************************************************/
/* Registers */
#define DigitalSet	123 // U16	DOMask	Sets output
#define DigitalClear	124 // U16	DOMask	Clears output
#define PwmStart	125 // U8	PwmMask	Starts pwm
#define PwmStop	126 // U8	PwmMask	Stops pwm
#define Heartbeat	12 // U8		Heartbeat event



In [42]:
txt_fnc = lambda row: f"""#define {row.Identifier}	{row.BitMask if "<<" not in row.BitMask else f"({row.BitMask})"}	// {row.Format}	{row.Description if pd.isnull(row.Description) is False else ""}
"""
mask_data = masks.apply(txt_fnc, axis=1).values.tolist()

header_mask =\
"""/************************************************************************/
/* Registers' bits                                                      */
/************************************************************************/
""" + f"{''.join(mask_data)}"

print(header_mask)


/************************************************************************/
/* Registers' bits                                                      */
/************************************************************************/
#define DOMask_Poke0	(1<<0)	// U16	Output of Poke0
#define DOMask_Poke1	(1<<1)	// U16	Output of Poke1
#define DOMask_Poke2	(1<<2)	// U16	Output of Poke2
#define PwmMask_DO0	(1<<0)	// U8	Pwm channel 0
#define PwmMask_DO1	(1<<1)	// U8	Pwm channel 1



In [44]:
f = open("header.h", "w")
f.writelines([header_register_s, "\n", header_register, "\n", header_mask])
f.close()
